In [49]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import torch
import torchvision
import torchvision.transforms as T

BATCH_SIZE = 4

IMG_SIZE = (256,256)

In [50]:
#!python download_data.py

In [51]:
filtered_categories = ["Beverages", "Sweet snacks", "Dairies", "Cereals and potatoes", "Meats", "Fermented foods", "Fermented milk products", 
                    "Groceries", "Meals", "Cereals and their products", "Cheeses", "Sauces", "Spreads", "Confectioneries", "Prepared meats", 
                    "Frozen foods", "Breakfasts", "Desserts", "Canned foods", "Seafood", "Cocoa and its products", "Fats", "Condiments", 
                    "Fishes", "Breads", "Yogurts", "Cakes", "Biscuits", "Pastas", "Legumes"]
filtered_categories = [s.lower() for s in filtered_categories]

CLASS_TO_INDEX = {filtered_categories[i]:i for i in range(len(filtered_categories))}
INDEX_TO_CLASS = {i:filtered_categories[i] for i in range(len(filtered_categories))}

In [52]:
transforms = T.Compose([T.Resize(IMG_SIZE), T.ToTensor()])

In [53]:
def one_hot_encoding(row):
    # Since we are reading a coma separated file for labels, the different labels are splitted with ; rather than , 
    m = len(filtered_categories)
    y = torch.zeros(m, dtype=float)
    file_labels = [int(l) for l in row["labels"].split(';')]
    for i in file_labels:
        y[i] = 1.0
    return y


class MultilabelDataset(torch.utils.data.Dataset):
    def __init__(self, split_folder_root, transform=None):
        super().__init__()
        self.transforms = transform
        self.img_paths = [os.path.join(split_folder_root, "images", f) for f in os.listdir(os.path.join(split_folder_root, "images"))]
        self.df_label = pd.read_csv(os.path.join(split_folder_root, "labels.csv"))


    def __getitem__(self, idx):
        img = plt.imread(self.img_paths[idx])
        img = Image.fromarray(img)
        row = self.df_label.loc[idx]
        y = one_hot_encoding(row)
        if self.transforms is not None:
            X = self.transforms(img)


        return X, y

    def __len__(self):
        return len(self.img_paths)

In [54]:
train_set = MultilabelDataset(os.path.join(os.path.abspath(os.path.curdir), "data", "train"), transform=transforms)
test_set = MultilabelDataset(os.path.join(os.path.abspath(os.path.curdir), "data", "test"), transform=transforms)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=0)

In [55]:
for i, data in enumerate(train_loader):
    # Print first batch just for example:
    if i == 0:
        img, label = data
        print(img)
        print(label)
        break

tensor([[[[0.3843, 0.3608, 0.3216,  ..., 0.3922, 0.3922, 0.4549],
          [0.3216, 0.2980, 0.2706,  ..., 0.3961, 0.3961, 0.4627],
          [0.2706, 0.2510, 0.2275,  ..., 0.3961, 0.3961, 0.4627],
          ...,
          [0.5451, 0.6000, 0.6314,  ..., 0.4941, 0.4980, 0.5490],
          [0.6392, 0.6667, 0.6627,  ..., 0.3843, 0.3882, 0.4706],
          [0.6667, 0.6980, 0.6941,  ..., 0.3412, 0.3765, 0.4863]],

         [[0.1490, 0.1294, 0.1059,  ..., 0.2549, 0.2510, 0.3137],
          [0.1098, 0.0941, 0.0706,  ..., 0.2549, 0.2588, 0.3216],
          [0.0824, 0.0706, 0.0549,  ..., 0.2510, 0.2588, 0.3216],
          ...,
          [0.5569, 0.6078, 0.6353,  ..., 0.3569, 0.3569, 0.4078],
          [0.6078, 0.6275, 0.6118,  ..., 0.2549, 0.2549, 0.3333],
          [0.5961, 0.6235, 0.6039,  ..., 0.2157, 0.2431, 0.3529]],

         [[0.1882, 0.1686, 0.1412,  ..., 0.1804, 0.1804, 0.2510],
          [0.1412, 0.1216, 0.0980,  ..., 0.1804, 0.1843, 0.2549],
          [0.1059, 0.0902, 0.0745,  ..., 0